In [ ]:
# Simple sanity check
import htcondor
print("HTCondor Collector:", htcondor.Collector().evaluate("Name"))

In [ ]:
#%pip install htcondor ipywidgets

In [ ]:
import htcondor
import ipywidgets as widgets
from IPython.display import display

ModuleNotFoundError: No module named 'htcondor'